In [16]:
import torch
import torch.nn as nn
import torchvision 
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

class DoubleConv(nn.Module):
    def __init__(self , in_channels , out_channels):
        super().__init__()
        
        self.conv = torch.nn.Sequential(
            nn.Conv2d(in_channels , out_channels, kernel_size=3 , stride= 1 , padding=1,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels , out_channels=out_channels , kernel_size = 3, stride=1 , padding = 1 , bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    def forward(self , x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self , in_channels=3 , out_channels=1 , features = [64,128,256,512]):
        super().__init__()
        self.in_channels = in_channels
        self.donws = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.maxpoll = nn.MaxPool2d(kernel_size=2)
        
        
        # Down sampling part 
        for feature in features:
            self.donws.append(DoubleConv(self.in_channels , feature))
            self.in_channels = feature
        
        # Up samping part
        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(feature*2 , feature , kernel_size=2 ,stride = 2))
            self.ups.append(DoubleConv(feature*2 , feature))
        
        # Bottelneck
        self.bottleneck = DoubleConv(features[-1] , features[-1]*2)
        
        # Final conv layer(1x1)
        self.fianl_conv = nn.Conv2d(features[0], out_channels, kernel_size=1 , stride=1)
        
        
    def forward(self , x):
        skip_connections = []
        # Down sapling part
        for down in self.donws:
            x = down(x)
            skip_connections.append(x)
            x = self.maxpoll(x)
        # Bottelencek
        x = self.bottleneck(x)
        
        skip_connections = skip_connections[::-1]
        # up sampling 
        for idx in range(0,len(self.ups),2): # 짝수 번째에서 up samping , 홀수 번째에 DoubleCOnv
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]
            if x.shape != skip_connection.shape:
                x = TF.resize(x, skip_connection.shape[2:] , antialias=True)
            # (B,C,H,W)에서 채널에 대하여 concat연산
            concat_x = torch.cat((skip_connection , x) ,dim=1)
            x = self.ups[idx+1](concat_x)
        
        return self.fianl_conv(x)

In [5]:
import matplotlib.pyplot as plt
from PIL import Image

In [17]:
from dataset import UNetDataset
traindataset = UNetDataset()
_, mask = traindataset[0]

In [22]:
mask[mask == 255] = 1
mask_t = torch.tensor(mask , dtype=torch.float).unsqueeze(0)
mask_t.shape

torch.Size([1, 1280, 1918])

In [26]:
from tqdm import tqdm
import time
for i in tqdm(range(100), desc = "Process Bar" , unit = " epoch"):
    time.sleep(0.01)


Process Bar: 100%|██████████| 100/100 [00:01<00:00, 59.45 epoch/s]


In [28]:
for i in tqdm(range(100)):
    time.sleep(0.1)

100%|██████████| 100/100 [00:10<00:00,  9.12it/s]


In [42]:
from tqdm import tqdm
import time

# scale 인자를 사용하여 자동 스케일링을 조절
with tqdm(total=10, unit="items", unit_scale=True, unit_divisor=10,colour="") as pbar:
    for i in range(10):
        time.sleep(0.1)
        pbar.update(1)

100%|██████████| 10.0/10.0 [00:01<00:00, 9.18items/s]


In [ ]:
from tqdm import tqdm
from pathlib import Path
def train_fn(model , loader , optimizer , loss_fn , device):
    model.train()
    train_loss = 0
    train_acc = 0
    loop = tqdm(loader ,desc = "Traning" , unit = "iter")
    for x,y in loader :
        x =x.to(device)
        y = y.to(device)
        pred = model(x)    
        loss = loss_fn(pred , y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # caculate metric(train_acc , loss)
        train_loss+=loss.item()
        train_acc += ((torch.argmax(pred , dim=1)==y).sum().item() / len(y))
        # loop update
        loop.update(1)
        
    train_loss /= len(loader)
    train_acc /= len(loader)
    loop.close()
    return train_loss , train_acc

def test_fn(model , loader , loss_fn , device):
    model.eval()
    test_loss =0
    test_acc = 0
    with torch.inference_mode():
        loop = tqdm(loader ,desc = "Test" , unit = "iter")        
        for x ,y in loader:
            x =x.to(device)
            y = y.to(device)
            pred = model(x)    
            test_loss += loss_fn(pred , y).item()
            test_acc += (torch.argmax(pred,dim=1)==y).sum() / len(y)

            # loop update
            loop.update(1)
        loop.close()
    test_loss /= len(loader)
    test_acc /= len(loader)
    return test_loss , test_acc

def train(model,
          train_loader,
          val_loader,
          optimizer,
          loss_fn,
          device,
          epochs):
    best_loss = None
    for epoch in range(epochs):
        train_loss , train_acc = train_fn(model, train_loader , optimizer , loss_fn , device)
        val_loss , val_acc = test_fn(model , val_loader , loss_fn , device)
        # print result
        print(f"[{epoch+1}/{epochs}] | Train_loss: {train_loss:.5f} , Train_acc : {train_acc*100:.2f}% \
                | Val_loss: {val_loss:.5f} , Val_acc : {val_acc*100:.2f}%")
        # model save
        if best_loss is None:
            best_loss = train_loss
            save_checkpoint(model , optimizer , "best" , "/model" , epoch = epoch , loss = train_loss , acc = train_acc)
        elif best_loss > train_loss:
            print("Best!!")
            best_loss = train_loss
            save_checkpoint(model , optimizer , "best" , "/model" , epoch = epoch , loss = train_loss , acc = train_acc)
        else:
            save_checkpoint(model , optimizer , "last" , "/model" , epoch = epoch , loss = train_loss , acc = train_acc)
            
            
        
        
        
def save_checkpoint(model , optimizer, name , path , **results):
    check_point = {}
    check_point.update({"model_state_dict" : model.state_dict()})
    check_point.update({"optimizer_state_dict" : optimizer.state_dict()})
    check_point.update(results)
    model_dir = Path(path)
    if model_dir.is_dir() and not model_dir.exists():
        model_dir.mkdir(parents=True , exist_ok=True)
        
    model_path = model_dir / str(name+".pth")
    torch.save(check_point, model_path)

def load_checkpoint(path , model , optimizer):
    check_point = torch.load(path)
    model.load_state_dict(check_point['model_state_dict'])
    optimizer.load_state_dict(check_point['optimizer_state_dict'])
        
        
    
            
            
            

In [54]:

for i in range(10):
    for  j in tqdm(range(10) , position=None):
        pass

100%|██████████| 10/10 [00:00<?, ?it/s]


In [59]:
import math
math.sin(1/math.pow(10000,4/6))

0.0021544330233656045

In [13]:
import torch
a = torch.tensor(range(24)).reshape(4,-1)
print(a)
torch.einsum("ij->i",a)


tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]])


tensor([ 15,  51,  87, 123])

In [ ]:
import zipfile
from pathlib import Path
from tqdm import tqdm
data_dir = Path("/kaggle/input/dogs-vs-cats")
with zipfile.ZipFile(data_dir/"train.zip") as zip_ref:
    files = zip_ref.infolist()
    for file in tqdm(files):
        zip_ref.extract(file , path=data_dir)

In [ ]:
import pandas as pd

a = pd.DataFrame()
a.plot()

In [ ]:
from sklearn.model_selection import train_test_split

train_test_split()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
from PIL import Image
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
class CatDogDataset(Dataset):
    def __init__(self ,home_dir, images , transform = None):
        self.images = images
        self.transform = transform
        self.class_to_index = {"cat" : 0 ,"dog" : 1}
        self.home_dir = Path(home_dir)
    def __len__(self):
        return len(self.images)
    def __getitem__(self , idx):
        img = Image.open(self.home_dir.joinpath(self.images[idx]))
        
        if self.transform is not None:
            ag = self.transform(image=img)
            img = ag["image"]
        label = self.class_to_index[self.images[idx].split(".")[0]]
        label = torch.FloatTensor(label)
        return img , label
def get_dataloader(home_dir , train_image , val_image , batch_size , train_transform = None , val_transform = None):
    train_dataset = CatDogDataset(home_dir , train_image , train_transform)
    val_dataset = CatDogDataset(home_dir , val_image , val_transform)
    train_loader = DataLoader(train_dataset , batch_size = batch_size , shuffle=True )
    test_loader = DataLoader(val_dataset , batch_size=batch_size , shuffle=False)
    return train_loader , test_loader

# Setting transform
train_transform = A.Compose( [ 
    A.Resize(240,240z),
    A.OneOf([
        A.HorizontalFlip(p=0.3),
        A.Rotate(limit = 20 ,p=0.5 , value = 0)
    ],p=0.5),
    
    
    A.Sharpen(p=0.3, lightness=(0.9,1.0)),
    A.Normalize(mean =[0.485, 0.456, 0.406] , std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])
val_transform = A.Compose( [ 
    A.Normalize(mean =[0.485, 0.456, 0.406] , std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

train_loader , val_loader = get_dataloader("./train" , train_img , val_img , 32 , train_transform , val_transform)
img , label = next(iter(train_loader))
img.shape , label.shape
        
        

torch.Size([3, 256, 28, 28])

In [65]:
import torch
import torch.nn as nn

class InceptionModule(nn.Module):
    def __init__(self , in_channels , ch1x1 , ch3x3_reduct , ch3x3 , ch5x5_reduct , ch5x5 , pool ):
        super(InceptionModule , self).__init__()
        
        self.conv1x1 = nn.Conv2d(in_channels , ch1x1 , kernel_size=1)
        self.conv3x3 = nn.Sequential(BaseConv2d(in_channels , ch3x3_reduct , kernel_size=1),
                                     BaseConv2d(ch3x3_reduct,ch3x3 , kernel_size=3,padding=1))
        self.conv5x5 = nn.Sequential(BaseConv2d(in_channels , ch5x5_reduct , kernel_size=1),
                                     BaseConv2d(ch5x5_reduct , ch5x5 , kernel_size=5 , padding=2))
        self.pool = nn.Sequential(nn.MaxPool2d(kernel_size=3,padding=1,stride=1),
                                  nn.Conv2d(in_channels , pool , kernel_size=1))
    def forward(self ,x ):
        x1 = self.conv1x1(x)
        x2 = self.conv3x3(x)
        x3 = self.conv5x5(x)
        x4 = self.pool(x)
        
        return torch.cat([x1,x2,x3,x4] , dim=1)
    

class BaseConv2d(nn.Module):
    def __init__(self , in_channels , out_channels  , **kwards):
        super(BaseConv2d, self).__init__()
        self.conv2d = nn.Conv2d(in_channels , out_channels  , **kwards)
        self.relu = nn.ReLU(inplace=True)
    def forward(self , x):
        return self.relu(self.conv2d(x))

class AuxMoudle(nn.Module):
    def __init__(self , in_channels , num_classes):
        super(AuxMoudle,self).__init__()
        self.aux = nn.Sequential(nn.AvgPool2d(kernel_size=5 , stride=3),
                                 BaseConv2d(in_channels,128,kernel_size = 1),
                                 nn.Flatten(start_dim=1),
                                 nn.Linear(4*4*128,1024),
                                 nn.ReLU(inplace=True),
                                 nn.Dropout(p=0.7),
                                 nn.Linear(1024,num_classes))
    def forward(self, x):
        return self.aux(x)

# image size =(3x224x224) 
class GoogleNet(nn.Module):
    def __init__(self, in_channels , num_classes):
        super().__init__()
        self.training =True
        self.num_classes = num_classes
        self.conv1 = nn.Sequential(BaseConv2d(3,64,kernel_size = 7,stride=2,padding=3),
                                 nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
                                 nn.LocalResponseNorm(2))
        self.conv2 = nn.Sequential(BaseConv2d(64,192,kernel_size=1),
                                 BaseConv2d(192,192,kernel_size=3,padding=1),
                                 nn.LocalResponseNorm(2),
                                 nn.MaxPool2d(kernel_size=3,stride=2,padding=1))
        self.inception3_a = InceptionModule(192,64,96,128,16,32,32)
        self.inception3_b = InceptionModule(256,128,128,192,32,96,64)
        self.aux1 = AuxMoudle(512,self.num_classes)
        self.inception4_a = InceptionModule(480,192,96,208,16,48,64)
        self.inception4_b = InceptionModule(512,160,112,224,24,64,64)
        self.inception4_c = InceptionModule(512,128,128,256,24,64,64)
        self.aux2 = AuxMoudle(528,self.num_classes)
        self.inception4_d = InceptionModule(512,112,144,288,32,64,64)
        self.inception4_e = InceptionModule(528,256,160,320,32,128,128)
        self.inception5_a = InceptionModule(832,256,160,320,32,128,128)
        self.inception5_b = InceptionModule(832,384,192,384,48,128,128)

        self.maxpool = nn.MaxPool2d(3,stride=2,padding=1)
        self.avgpool = nn.AvgPool2d(7)
        self.fc = nn.Sequential(nn.Flatten(start_dim=1),
                                nn.Dropout(p=0.4),
                                nn.Linear(1024,num_classes))
    
    def forward(self , x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.inception3_a(x)
        x = self.inception3_b(x)
        x = self.maxpool(x)
        x = self.inception4_a(x)
        x = self.inception4_b(x)
        
        if self.training:
            out1 = self.aux1(x)
        x = self.inception4_c(x)
        x = self.inception4_d(x)
        if self.training:
            out2 = self.aux2(x)
        x = self.inception4_e(x)
        x = self.maxpool(x)
        x = self.inception5_a(x)
        x = self.inception5_b(x)
        x = self.avgpool(x)
        x = self.fc(x)
        if self.training:
            return [x , out1 , out2]
        else:
            return x


    

In [68]:
from torchinfo import summary
model =  GoogleNet(3,1000)
Batch_size = 16
summary(model , input_size = (Batch_size, 3,224,224))

Layer (type:depth-idx)                   Output Shape              Param #
GoogleNet                                [16, 1000]                6,379,728
├─Sequential: 1-1                        [16, 64, 56, 56]          --
│    └─BaseConv2d: 2-1                   [16, 64, 112, 112]        --
│    │    └─Conv2d: 3-1                  [16, 64, 112, 112]        9,472
│    │    └─ReLU: 3-2                    [16, 64, 112, 112]        --
│    └─MaxPool2d: 2-2                    [16, 64, 56, 56]          --
│    └─LocalResponseNorm: 2-3            [16, 64, 56, 56]          --
├─Sequential: 1-2                        [16, 192, 28, 28]         --
│    └─BaseConv2d: 2-4                   [16, 192, 56, 56]         --
│    │    └─Conv2d: 3-3                  [16, 192, 56, 56]         12,480
│    │    └─ReLU: 3-4                    [16, 192, 56, 56]         --
│    └─BaseConv2d: 2-5                   [16, 192, 56, 56]         --
│    │    └─Conv2d: 3-5                  [16, 192, 56, 56]         331,

In [69]:
m = GoogleNet(3,1000)
x, out1 , out2 = m(torch.randn(16,3,224,224))
x.shape, out1.shape, out2.shape

(torch.Size([16, 1000]), torch.Size([16, 1000]), torch.Size([16, 1000]))

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters() , lr = 1e-4)
torch.nn.BCEWithLogitsLoss